In [1]:
%matplotlib inline
# %matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import xarray as xr

from pyxlma.lmalib.traversal import OneToManyTraversal

from tracerpaws.tracktimeseries import open_track_timeseries_dataset, track_polarimetry, add_track_durations, feature_distance_from



In [3]:
track_file = '/home/jovyan/efs/tracer/NEXRAD/tobac_Save_20220806/Track_features_merges.nc'
timeseries_file = '/home/jovyan/efs/tracer/NEXRAD/tobac_Save_20220806/timeseries_data_melt4400.nc'
ref_grid = '/home/jovyan/efs/tracer/NEXRAD/20220827/KHGX20220827_000355_V06_grid.nc'

In [4]:
combo = open_track_timeseries_dataset(track_file, timeseries_file, ref_grid)

In [5]:
combo

<xarray.Dataset>
Dimensions:                           (feature: 29009, time: 210, y: 1001,
                                       x: 1001, track: 1307, cell: 1531)
Coordinates:
  * feature                           (feature) int64 1 2 3 ... 29008 29009
  * time                              (time) datetime64[ns] 2022-08-06T00:05:...
  * y                                 (y) float64 -2.5e+05 ... 2.5e+05
  * x                                 (x) float64 -2.5e+05 ... 2.5e+05
  * track                             (track) float64 0.0 1.0 ... 1.306e+03
  * cell                              (cell) int64 -1 12 14 ... 10767 10768
Data variables: (12/48)
    feature_time_index                (feature) int64 0 0 0 0 ... 209 209 209
    feature_id_this_frame             (feature) int64 ...
    feature_hdim1_coordinate          (feature) float64 13.36 0.9971 ... 977.4
    feature_hdim2_coordinate          (feature) float64 495.5 730.4 ... 117.3
    feature_grid_cell_count           (feature) int64 ...
    feature_threshold_max             (feature) float64 ...
    ...                                ...
    feature_rhvdeficitwt_total        (feature) float64 nan nan nan ... nan nan
    feature_flash_count               (feature) float64 nan nan nan ... nan nan
    feature_flash_count_area_LE_4km   (feature) float64 nan nan nan ... nan nan
    feature_flash_count_area_GT_4km   (feature) float64 nan nan nan ... nan nan
    feature_longitude_center          (feature) float64 -95.1 -93.92 ... -97.1
    feature_latitude_center           (feature) float64 27.28 27.22 ... 31.6

Look at Zdrcolwt to see if we can reproduce count deficit on that file by using various criteria. We know non-norm vars are eliminated for some reason. Also look at just one file in plot_track_histo's bottom section to get an exact match.

In [10]:
def check_var(a):
    nonzero = (a > 0.0)
    finite = np.isfinite(a)
    nan = np.isnan(a)
    real = nonzero & finite
    inf = nonzero &(~finite)
    pos_finite = finite & nonzero
    neg_finite = finite & (a < 0)
    zero = np.isclose(a, 0.0)
    print(a.shape[0], nonzero.sum(), finite.sum(), nan.sum(), inf.sum(), real.sum(), pos_finite.sum(), neg_finite.sum(), zero.sum())

In [11]:
check_var(combo.feature_area)

29009 <xarray.DataArray 'feature_area' ()>
array(27442) <xarray.DataArray 'feature_area' ()>
array(29009) <xarray.DataArray 'feature_area' ()>
array(0) <xarray.DataArray 'feature_area' ()>
array(0) <xarray.DataArray 'feature_area' ()>
array(27442) <xarray.DataArray 'feature_area' ()>
array(27442) <xarray.DataArray 'feature_area' ()>
array(0) 1567


In [12]:
check_var(combo.feature_kdpwt_total)

29009 <xarray.DataArray 'feature_kdpwt_total' ()>
array(941) <xarray.DataArray 'feature_kdpwt_total' ()>
array(9104) <xarray.DataArray 'feature_kdpwt_total' ()>
array(19905) <xarray.DataArray 'feature_kdpwt_total' ()>
array(0) <xarray.DataArray 'feature_kdpwt_total' ()>
array(941) <xarray.DataArray 'feature_kdpwt_total' ()>
array(941) <xarray.DataArray 'feature_kdpwt_total' ()>
array(0) 8163


In [13]:
check_var(combo.feature_zdrwt_total)

29009 <xarray.DataArray 'feature_zdrwt_total' ()>
array(5349) <xarray.DataArray 'feature_zdrwt_total' ()>
array(9104) <xarray.DataArray 'feature_zdrwt_total' ()>
array(19905) <xarray.DataArray 'feature_zdrwt_total' ()>
array(0) <xarray.DataArray 'feature_zdrwt_total' ()>
array(5349) <xarray.DataArray 'feature_zdrwt_total' ()>
array(5349) <xarray.DataArray 'feature_zdrwt_total' ()>
array(0) 3755
